(실습-10) RAG 실습
실습 개요
1) 실습 목적
이번 실습에서는 ColBERT와 LLM을 이용하여 RAG를 구현해 본다.
RAG 구현을 위해 LLM 프롬프트를 어떻게 사용할 수 있는지 확인한다.
2) 수강 목표

RAG 구현을 위한 내부 모듈 및 시스템 파이프라인을 이해한다.
ColBERT와 OpenAI API를 사용하여 RAG를 실제로 구현할 수 있다.
Function calling을 포함한 프롬프트 엔지니어링 방법에 대해 이해한다.
실습 목차
RAG 개념 이해
검색엔진 준비 - ColBERT
대화형 IR
데이터셋 개요
데이터셋: LoTTE 벤치마크의 dev 세트
데이터셋 개요
LoTTE: ColBERT에서 소개하는 공개 데이터 셋이다.
데이터셋 라이센스
LoTTE: MIT
LoTTE 데이터셋 상세 설명
LoTTE 데이터 세트는 Wikipedia와 같은 엔티티 중심의 지식 기반에서는 잘 다루지 않을 수 있는 롱테일 주제에 대한 IR 시스템을 평가하기 위해 설계되었습니다. 이번 실습에서는 주로 반려 동물을 다루고 있는 문서를 사용합니다.
환경 설정
OpenAI 패키지를 설치한다.
RAG용 검색엔진으로 사용할 ColBERT 패키지를 설치한다.

In [1]:
!pip install openai

In [2]:
!pip install "colbert-ir[faiss-gpu, torch]"

  Obtaining dependency information for colbert-ir[faiss-gpu,torch] from https://files.pythonhosted.org/packages/24/dd/3f22df40526cb4ae7263879449db46c55c541d10617456ddc5894d12e4f5/colbert_ir-0.2.14-py3-none-any.whl.metadata
  Obtaining dependency information for bitarray from https://files.pythonhosted.org/packages/e4/37/df70f878b8e4dbf95558eea8c7611db8bf14b0064992078c36339c529b84/bitarray-2.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/60/2d/963b266bb8f88492d5ab4232d74292af8beb5b6fdae97902df9e284d4c32/datasets-2.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for flask from https://files.pythonhosted.org/packages/61/80/ffe1da13ad9300f87c93af113edd0638c75138c42a0994becfacac078c06/flask-3.0.3-py3-none-any.whl.metadata
  Obtaining dependency information for git-python from https://files.pythonhosted.org/packages/8a/de/0cc6353a45cdb1e137cffac5383097b300cc

In [9]:
from datasets import load_dataset

dataset = 'lifestyle'
datasplit = 'dev'

collection_dataset = load_dataset("colbertv2/lotte_passages", dataset)
collection = [x['text'] for x in collection_dataset[datasplit + '_collection']]

queries_dataset = load_dataset("colbertv2/lotte", dataset)
queries = [x['query'] for x in queries_dataset['search_' + datasplit]]

print(f'Loaded {len(queries)} queries and {len(collection):,} passages')
print(queries[0])
print(collection[0])

Generating search_test split: 100%|██████████| 661/661 [00:00<00:00, 329189.62 examples/s]

Loaded 417 queries and 268,893 passages
how much should i feed my 1 year old english mastiff?
In my experience rabbits are very easy to housebreak. They like to pee and poop in the same place every time, so in most cases all you have to do is put a little bit of their waste in the litter box and they will happily use the litter box. It is very important that if they go somewhere else, miss the edge or kick waste out of the box that you clean it up well and immediately as otherwise those spots will become existing places to pee and poop. When you clean the box, save a little bit of waste and put it in the cleaned box so it smells right to them. For a more foolproof method, you can get a piece of wood soaked with their urine and put that in the box along with droppings or cage them so that they are only in their litter box for a week. Generally, if I try the first method and find that they are not using only the box on the first day, I go for the litter box only for a week method. The wo

## 1. RAG 개념 이해

레퍼런스 정보가 있다고 가정하고 극 정보를 토대로 질문에 답하는 예시

In [24]:
# OpenAI 라이브러리 및 필요한 모듈을 가져옵니다.

import os
import json
from openai import OpenAI

from dotenv import load_dotenv

load_dotenv()

OPENAI_KEY = os.getenv('OPENAI_KEY')

os.environ["OPENAI_API_KEY"] = OPENAI_KEY

client = OpenAI()

llm_model = "gpt-3.5-turbo-1106"


In [19]:
messages = [
    {'role' : "user", "content": "너는 누구니"},
    {"role" : "assistant", "content": "저는 시강 검색 도우미 Good Place 라고 합니다."},
    {"role" : "user" ,"content": "강남역 근처 태국 음식 추천해줘"}
]

reference =[
    {"식당명": "파파야리프", "대표 메뉴": "똠양꿍", "분위기": "이국적/이색적", "기타": "데이트에 적합"},
    {"식당명": "할랄가이즈", "대표 메뉴": "할랄", "분위기": "무슬림, 이국적", "기타":"가성비"},
    {"식당명": "인더비엣", "대표 메뉴": "팟타야, 쌀국수", "분위기": "깔끔", "기타": "해장"}
    
]

persona = """

## Name : Good Place

## Role : 식당 검색 도우미

## Instruction

- 사용자의 이전 메시지 정보 및 주어진 검색 결과 (JSON 형태로 제공) 정보를 활용하여 간결하게 답변을 생성한다.
- 주어진 검색 결과 정보로 대답할 수 없는 경우는 정보가 부족해서 답을 할 수 없다고 대답한다.
- 사용자가 상요한 언어로 답변을 생성한다.


"""

In [25]:
# 레퍼런스 없이 답변 생성하는 경우 - hallucinaion 발생

# 그러니까 학습을 진행하지 않고 그냥 자체 gpt로 응답을 할 때 타이쇼라는 음식점은 존재하지만 쏘파 라는 은식점은 존재하지 않는다.

result = client.chat.completions.create(
    model=llm_model,
    messages=messages,
    temperature=0,
    seed=1
)

print(result.choices[0].message)

ChatCompletionMessage(content='강남역 근처에서 태국 음식을 즐길 수 있는 몇 가지 좋은 식당이 있습니다. "타이 오디엔스"는 고품질의 태국 음식을 제공하며, "쏭타이"는 정통 태국 요리를 경험할 수 있는 곳입니다. 또한 "쏭카오"는 분위기 좋은 태국 레스토랑으로 유명합니다. 이 식당들 중 하나를 선택해보시기 바랍니다!', role='assistant', function_call=None, tool_calls=None)


In [ ]:
content = {
    '이전 메시지 정보': messages,
    '검색결과':
}